In [803]:
### correct One

import pandas as pd
import warnings
warnings.filterwarnings("ignore")

df=pd.read_csv(r"C:\Users\sakth\Downloads\fraudulent_claims_dataset_1000.csv")

In [805]:
df

,Claim_ID,Claim_Date,Customer_ID,Claim_Amount,Claim_Type,Suspicious_Flags,Fraud_Label
0,C0001,2024-02-01,P8674,9740.26,Medical,1,0
1,C0002,2024-02-16,P7122,2424.50,Medical,1,1
2,C0003,2024-05-07,P1481,9177.37,Life,0,0
3,C0004,2023-03-20,P1153,9884.64,Home,0,0
4,C0005,2023-03-13,P8617,19996.01,Home,0,0
...,...,...,...,...,...,...,...
995,C0996,2024-01-30,P8321,32391.14,Home,0,0
996,C0997,2024-10-28,P2828,21381.25,Auto,0,0
997,C0998,2023-06-21,P8909,38194.38,Auto,0,0
998,C0999,2024-02-23,P6588,30871.92,Home,0,0


In [807]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Claim_ID          1000 non-null   object 
 1   Claim_Date        1000 non-null   object 
 2   Customer_ID       1000 non-null   object 
 3   Claim_Amount      1000 non-null   float64
 4   Claim_Type        1000 non-null   object 
 5   Suspicious_Flags  1000 non-null   int64  
 6   Fraud_Label       1000 non-null   int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 54.8+ KB


In [809]:
df.drop(columns=['Claim_ID','Claim_Date','Customer_ID'],inplace=True)

In [811]:
df.isnull().sum()

Claim_Amount        0
Claim_Type          0
Suspicious_Flags    0
Fraud_Label         0
dtype: int64

In [813]:
df=df[['Claim_Amount','Claim_Type','Suspicious_Flags','Fraud_Label']]

In [815]:
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder()
claim=ohe.fit_transform(df[['Claim_Type']]).toarray()
claim_df=pd.DataFrame(claim,columns=ohe.get_feature_names_out(['Claim_Type'])).astype(int)
df=df.drop(columns=['Claim_Type']).reset_index(drop=True)
df=pd.concat([df,claim_df],axis=1)

In [817]:
df['Claim_Amount']=df['Claim_Amount'].astype("int64")

In [819]:
df.head(10)


,Claim_Amount,Suspicious_Flags,Fraud_Label,Claim_Type_Auto,Claim_Type_Home,Claim_Type_Life,Claim_Type_Medical
0,9740,1,0,0,0,0,1
1,2424,1,1,0,0,0,1
2,9177,0,0,0,0,1,0
3,9884,0,0,0,1,0,0
4,19996,0,0,0,1,0,0
5,20714,0,0,1,0,0,0
6,43838,1,1,0,0,0,1
7,25995,0,0,1,0,0,0
8,36446,0,0,0,0,1,0
9,6102,0,0,0,1,0,0


In [821]:
X=df[['Claim_Amount','Suspicious_Flags','Claim_Type_Auto',	'Claim_Type_Home'	,'Claim_Type_Life','Claim_Type_Medical']]


In [823]:
Y=df[['Fraud_Label']]

In [825]:
from  sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [827]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
svm = SVC()
model = LogisticRegression(class_weight='balanced',solver='liblinear')
model.fit(X_train,Y_train)


LogisticRegression(class_weight='balanced', solver='liblinear')

In [829]:
ypred=model.predict(X_test)
from sklearn.metrics import classification_report
fullreport=classification_report(Y_test,ypred)

In [831]:
print(fullreport)

              precision    recall  f1-score   support

           0       1.00      0.86      0.93       166
           1       0.60      1.00      0.75        34

    accuracy                           0.89       200
   macro avg       0.80      0.93      0.84       200
weighted avg       0.93      0.89      0.90       200



In [847]:
import numpy as np
inputdata=(2000,	1,	0,	0,	0,	1	)
inputdataarray=np.asarray(inputdata)
inputdatareshape=inputdataarray.reshape((1,-1))
prediction=model.predict(inputdatareshape)
print(prediction)

[1]


In [849]:
X.head(10)

,Claim_Amount,Suspicious_Flags,Claim_Type_Auto,Claim_Type_Home,Claim_Type_Life,Claim_Type_Medical
0,9740,1,0,0,0,1
1,2424,1,0,0,0,1
2,9177,0,0,0,1,0
3,9884,0,0,1,0,0
4,19996,0,0,1,0,0
5,20714,0,1,0,0,0
6,43838,1,0,0,0,1
7,25995,0,1,0,0,0
8,36446,0,0,0,1,0
9,6102,0,0,1,0,0


In [855]:
import pickle 
with open('fraudcorrect.pkl','wb')as file:
    pickle.dump(model,file)

In [857]:
print(model.feature_names_in_)


['Claim_Amount' 'Suspicious_Flags' 'Claim_Type_Auto' 'Claim_Type_Home'
 'Claim_Type_Life' 'Claim_Type_Medical']
